[![alt text](https://avatars1.githubusercontent.com/u/59831504?s=400&v=4 "MtheEPIC User Icon")](https://github.com/MtheEPIC/KaggleProj)

[![alt text](https://avatars1.githubusercontent.com/u/59831504?s=400&v=4 "MtheEPIC User Icon")](https://github.com/MtheEPIC/DataImaging)

# Import libs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
sns.axes_style("whitegrid")

# import keras
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
# from keras.optimizers import Adam
# from keras.callbacks import TensorBoard

# from sklearn.model_selection import train_test_split
# import tensorflow as tf

from sklearn.decomposition import PCA

from sklearn.preprocessing import MinMaxScaler, binarize
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score, plot_roc_curve, confusion_matrix, classification_report, precision_recall_curve, average_precision_score, cross_val_predict
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV

from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

ImportError: cannot import name 'cross_val_predict' from 'sklearn.metrics' (C:\Users\mthee\Anaconda3\lib\site-packages\sklearn\metrics\__init__.py)

---
# Data preparation

## load data

In [ ]:
train_df = pd.read_csv('data/fashion-mnist_train.csv')
test_df = pd.read_csv('data/fashion-mnist_test.csv')

In [ ]:
train_df.head()

0 is t shirt\
1 is trousers\
2 is pullover\
3 is dress\
4 is coat\
5 is sandals\
6 is shirt\
7 is sneaker\
8 is bag\
9 is ankle boots

In [ ]:
train_df.info()
test_df.info()

In [ ]:
train_df.describe()

## split the images from the labels

## split the training data into train and validate

In [ ]:
train_data = np.array(train_df, dtype='float32')
test_data = np.array(test_df, dtype='float32')

X_train = train_data[:, 1:] / 1
y_train = train_data[:, 0]

X_test = test_data[:, 1:] / 1#/128#/255
y_test = test_data[:, 0]

In [ ]:
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio_)
x = np.arange(1, len(cumsum)+1)
plt.plot(x, cumsum)
plt.show()

In [ ]:
pca = PCA(n_components=0.90)
X_train_pca = pca.fit_transform(X_train)

# print(pca.explained_variance_ratio_)
# print(pca.singular_values_)

d = len(pca.singular_values_)
pca2 = PCA(n_components=d)
X_test_pca = pca2.fit_transform(X_test)
d

# run again with pca

In [ ]:
first_image = X_train[0]

first_image = np.array(first_image, dtype='uint8')
pixels = first_image.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
def paint():
    tmp = train_df.groupby('label').sum()
    train_data = np.array(tmp, dtype='float32')

    first_image = train_data[0]

    first_image = np.array(first_image, dtype='uint8')
    pixels = first_image.reshape((28, 28))
    import numpy as np; np.random.seed(0)

    import seaborn as sns; sns.set()

    # uniform_data = np.random.rand(10, 12)
    uniform_data = pixels#//500
    ax = sns.heatmap(uniform_data)

    uniform_data.max()

In [ ]:
# x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [ ]:
def evaluate(true, pred):
#     model_accuracy = roc_auc_score(true, pred, multi_class="ovr")
#     null_accuracy = roc_auc_score(true, true.replace(1, 0))
    
#     print("Null AUC Score: {:.5f}".format(null_accuracy))
#     print("Model AUC Score: {:.5f}".format(model_accuracy))

#     if null_accuracy >= model_accuracy:
#         print("The model isn't effective")
# #         return
#     else:
#         print("The model is better then a 'dumb' model")
    print("confusion_matrix:\n", confusion_matrix(true, pred))
    print(classification_report(true, pred))

In [ ]:
# len(predictions)
# y_test
# evaluate(y_test, predictions)

---
## Models

In [ ]:
model_dict = {}

### Naive Bayes

In [ ]:
gnb = GaussianNB()

gnb.fit(X_train_pca, y_train)
predictions = gnb.predict(X_test_pca)
evaluate(y_test, predictions)

scores = cross_val_score(gnb, X_train_pca, y_train, cv=5)
gnb_score = scores

model_dict.update({"GaussianNB": gnb_score})

print("Accuracy: %0.2f (+/- %0.2f)" % (gnb_score.mean(), gnb_score.std() * 2))

In [ ]:
# mnb = MultinomialNB()
# mnb.fit(X_train_pca, y_train)
# predictions = mnb.predict(X_test_pca)
# evaluate(y_test, predictions)

### Random Forest

In [ ]:
rf = RandomForestClassifier(random_state=0, max_depth=None, n_estimators=100)

# scores = cross_val_score(rf, X_train_pca, y_train, cv=5, scoring='neg_mean_squared_error')
# rf_score = np.sqrt(-scores)
scores = cross_val_score(rf, X_train_pca, y_train, cv=5, n_jobs=-1)
rf_score = scores

model_dict.update({"RandomForestClassifier": rf_score})

print("Accuracy: %0.2f (+/- %0.2f)" % (rf_score.mean(), rf_score.std() * 2))

tmp = cross_val_predict(rf, X_train_pca, y_train, cv=5, n_jobs=-1)


print(len(predictions))
print(len(tmp))
# y_test_appended = np.append(np.append(y_test,y_test),np.append(np.append(y_test, y_test),np.append(y_test, y_test)))
evaluate(y_train, tmp)

In [ ]:
predictions_train = rf.predict(X_train_pca)
evaluate(y_train, predictions_train)

In [ ]:
from sklearn.model_selection import cross_validate
tmp2 = cross_val_predict(rf, X_train_pca, y_train, cv=5, n_jobs=-1)
tmp2

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#now lets find the optimum K 
k_max = 20
k_min = 1
err_rate = []
for i in range(k_min, k_max+1):
    k = KNeighborsClassifier(n_neighbors=i)
#     k.fit(X_train_pca, y_train)
#     pred_i = k.predict(X_test_pca)
#     err_rate.append(np.mean(pred_i != y_test))
    scores = cross_val_score(k, X_train_pca, y_train, cv=5, n_jobs=-1)
    if scores.std() > 0.1:
        err_rate.append(0)
    else:
        err_rate.append(scores.mean())

In [ ]:
plt.plot(np.arange(k_min, k_max+1), err_rate)
plt.show()

In [ ]:
index = err_rate.index(min(err_rate)) + 1
index

In [ ]:
k = KNeighborsClassifier(n_neighbors=index)
# k.fit(X_train_pca, y_train)
# predictions = k.predict(X_test_pca)

In [ ]:
k_scores = cross_val_score(k, X_train_pca, y_train, cv=5)
model_dict.update({"KNN": k_scores})
print("Accuracy: %0.2f (+/- %0.2f)" % (k_scores.mean(), k_scores.std() * 2))

### Ada Boost

### Logistic Regression

In [ ]:
lr = LogisticRegression(random_state=0)#, solver='lbfgs', penalty='l2')
# lr2.fit(X_train_pca, y_train)
# predictions = lr2.predict(X_train_pca)
# evaluate(y_train, predictions)

lr_scores = cross_validate(lr2, X_train_pca, y_train, cv=5, n_jobs=-1)

model_dict.update({"logistic regression": lr_scores})

print("Accuracy: %0.2f (+/- %0.2f)" % (lr_scores['test_score'].mean(), lr_scores['test_score'].std() * 2))
print('the mean train time is {} seconds'.format(lr_scores['fit_time'].mean()))
print('the mean test time is {} seconds'.format(lr_scores['score_time'].mean()))
# lr2_score = scores
# print("Accuracy: %0.2f (+/- %0.2f)" % (lr2_score.mean(), lr2_score.std() * 2))

# tmp = cross_val_predict(lr2, X_train_pca, y_train, cv=5, n_jobs=-1)
# tmp

# print(len(predictions))
# print(len(tmp))
# # y_test_appended = np.append(np.append(y_test,y_test),np.append(np.append(y_test, y_test),np.append(y_test, y_test)))
# evaluate(y_train, tmp)


### SVC

In [ ]:
svc = SVC(random_state=0, max_iter=500)
# svc.fit(X_train, y_train)
# predictions = svc.predict(X_test)
# evaluate(y_test, predictions)

svc_scores = cross_validate(svc, X_train_pca, y_train, cv=5, n_jobs=-1)
model_dict.update({"SVC": svc_scores})
print("Accuracy: %0.2f (+/- %0.2f)" % (svc_scores['test_score'].mean(), svc_scores['test_score'].std() * 2))
print('the mean train time is {} seconds'.format(svc_scores['fit_time'].mean()))
print('the mean test time is {} seconds'.format(svc_scores['score_time'].mean()))


### Decision Tree

In [ ]:
dtc = DecisionTreeClassifier()
# dtc.fit(X_train, y_train)
# predictions = dtc.predict(X_test)
# evaluate(y_test, predictions)

dtc_scores = cross_validate(svc, X_train_pca, y_train, cv=5, n_jobs=-1)
model_dict.update({"decision tree": dtc_scores})
print("Accuracy: %0.2f (+/- %0.2f)" % (dtc_scores['test_score'].mean(), dtc_scores['test_score'].std() * 2))
print('the mean train time is {} seconds'.format(dtc_scores['fit_time'].mean()))
print('the mean test time is {} seconds'.format(dtc_scores['score_time'].mean()))


In [ ]:
def best_model():
    max_score = 0
    best_key = None
    for key in model_dict:
        model_score = model_dict[key].mean()
        if max_score < model_score:
            max_score = model_score
            best_key = key
    print('the best model is {} with the score of {}'.format(best_key, max_score))

In [ ]:
best_model()